In [ ]:
import pandas as pd
import datetime
from datetime import datetime

%store -r query_vars

# TODO deeper look at 
#'Number of deleted event pages'     and 
#'Number of disabled event registrations'

In [ ]:
query_vars

In [ ]:
#https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
#pd.read_csv(file, sep='\t', header=None, names=headers, dtype=dtypes, parse_dates=parse_dates)

parse_dates_editors = ['cep_registered_at', 'cep_unregistered_at','last_activity_date','ac_date']#'gu_registration'
parse_dates_df = ['event_last_edit','event_start_utc','event_end_utc','event_deleted_at','event_created_at']
file = 'output/handling/metrics.csv'

editors = pd.read_csv('output/handling/editors.csv',parse_dates=parse_dates_editors)
#editors['cep_registered_at'] = pd.to_datetime(editors['cep_registered_at'], errors='coerce')
#editors['cep_registered_at'] = pd.to_datetime(editors['cep_registered_at'], format = "%Y%m%d%H%M%S").dt.strftime("%Y%m%d%H")

df = pd.read_csv(file, parse_dates=parse_dates_df)

## People

In [ ]:
#Number of organizers who only create 1 page total within period
#number of unique usernames WHERE editors['editor_type']=='event_organizer' AND DISTINCT event_id ==1

#groupby_organizers_event_organized_count
o_group = editors.loc[(editors['editor_type']=='event_organizer')].groupby('username')
groupby_organizers_event_organized_count = pd.Series.to_frame(o_group.apply(lambda x: x['event_id'].nunique())).reset_index() #todo work with the series object instead of transforming to df...if fewer lines of code: https://stackoverflow.com/questions/35690499/pandas-select-by-bigger-than-a-value

In [ ]:
#Number of organizers that enabled registration by experience 
organizers_x_exp = pd.Series.to_frame(editors.loc[(editors['editor_type']=='event_organizer')].groupby('user_tenure_bucket')['username'].nunique()).reset_index().rename(columns={'username': 'count'})

In [ ]:
#unique participants on the tool
#unique organizer count on tool events

group = editors.groupby('editor_type')

unique_users_by_role = group.apply(lambda x: x['user_id'].nunique()).reset_index(name=query_vars['metrics_month']).rename({'editor_type': 'metric'}, axis='index')

#rename the column
mapping = {unique_users_by_role.columns[0]:'metric'}
unique_users_by_role = unique_users_by_role.rename(columns=mapping)
unique_users_by_role

# Analyze

In [ ]:
# Convert the string in query_vars['...'] to a datetime.date object
query_vars['api_metrics_month_first_day_object'] = datetime.strptime(query_vars['api_metrics_month_first_day'], '%Y%m%d').date()
query_vars['api_metrics_month_next_month_first_day_object'] = datetime.strptime(query_vars['api_metrics_month_next_month_first_day'], '%Y%m%d').date()

In [ ]:
calculations = {  
    'Unique events'                          : df['event_page_title'].nunique(),
    'New events this period'                 : df[df['event_created_at'].dt.date.between(pd.to_datetime(query_vars['utc_start']).date(), pd.to_datetime(query_vars['utc_end']).date())]['event_id'].nunique(),
    'Number of deleted event pages'          : df[df['event_deleted_at'].notnull()]['event_page_title'].nunique(),
    'Net events'                             : (df['event_page_title'].nunique())-(df[df['event_deleted_at'].notnull()]['event_page_title'].nunique()),
    'Number of disabled event registrations' : df[df['event_deleted_at'].dt.date.between(query_vars['api_metrics_month_first_day_object'], query_vars['api_metrics_month_next_month_first_day_object'])]['event_id'].nunique(),
    
    'pageviews'                              : df['pageviews'].sum(),
    
    'Events_w_chatgroup links'               : df[df['event_chat_url_clean'].notnull()]['event_page_title'].nunique(),
    'Event registrations with meeting URL'   : df[df['event_meeting_url'].notnull()]['event_page_title'].nunique(),
    
    
    'Unique participants': editors.loc[(editors['editor_type']=='event_participant')]['username'].nunique(),
    'Participants registered': editors[editors['cep_registered_at'].dt.date.between(pd.to_datetime(query_vars['utc_start']).date(), pd.to_datetime(query_vars['utc_end']).date())]['user_id'].nunique(),
    'Number of new accounts created' : df['new_users_count'].sum(),
    
    'Number of participants who canceled a registration': editors[editors['cep_unregistered_at'].notnull()]['username'].nunique(),
    
    'Unique organizers': editors.loc[(editors['editor_type']=='event_organizer')]['username'].nunique(),
    'Number of organizers who only create 1 page total within period': len(groupby_organizers_event_organized_count[groupby_organizers_event_organized_count[0] == 1 ]),
    '1_to_3 years organizers': organizers_x_exp['count'][0],
    '3_to_10 years organizers': organizers_x_exp['count'][1],
    '10+ years organizers': organizers_x_exp['count'][2]
    
    }


metrics_df = pd.DataFrame.from_dict(calculations, orient='index', columns=[query_vars['metrics_month']])

#rename index
metrics_df.index.names = ['metric']
metrics_df = metrics_df.reset_index()

In [ ]:
#metrics_df.pivot(columns='metric', values=)

metrics_df.set_index('metric',inplace=True)
final = metrics_df.transpose().reset_index()#.rename(columns={'metric':'metric_month'})

In [ ]:
final.to_csv('output/campaigns_metrics.csv', index=True)

## TODO By region

## Other